In [1]:
pwd

'E:\\Users\\Raphael\\Desktop\\Silent Night (Easy)'

In [2]:
home = _

In [3]:
#get some imports
import mido
from os.path import join as opj
import rtmidi
import xml.etree.ElementTree as ET

In [4]:
#enumerate available midi out ports
available_ports = rtmidi.MidiOut().get_ports()
available_ports

['Microsoft GS Wavetable Synth 0']

In [5]:
# get the first out port
port = available_ports[0]
port

'Microsoft GS Wavetable Synth 0'

In [6]:
# open the example file
mid = mido.MidiFile(opj(home,"Silent Night (Easy).mid"))
mid

<midi file 'E:\\Users\\Raphael\\Desktop\\Silent Night (Easy)\\Silent Night (Easy).mid' type 1, 2 tracks, 322 messages>

In [7]:
#helper to play and examine the song

def play_song():
    with mido.open_output(port) as output:
        for msg in mid.play():
            print(msg)
            data = msg.dict()
            if 'velocity' in data:
                vel = data['velocity']
                if vel > 0 and data['time'] == 0:
                    print('Note On')
            else:
                print('Not a note')
            output.send(msg)

In [8]:
#get the synthesia metadata
with open(opj(home,'Silent Night (Easy).synthesia')) as xml:
    tree = ET.parse(xml)
    root = tree.getroot()
    for item in root.findall('./Songs'):
        for child in item:
            fingerhints = child.attrib['FingerHints']
right,left = fingerhints.split(' t1: ')
right,left

('9097909700800788098787688098787699097800760766',
 '53131531315313153131531315313153131543215215531315432152155313153131531315313153131531315313153215313153131153131531315')

In [9]:
# do some decoding on their weird format
# 1,2,3,4,5 = left hand standard representation
# 0,6,7,8,9 = right hand with +5 offset except for thumb which is 0
def cvt_fingers(v):
    arr = []
    for cha in v:
        cha = int(cha)
        if cha > 5:
            cha -=5
        elif cha == 0:
            cha = 5
        arr += [cha]
    return arr

In [10]:
left,right = cvt_fingers(left),cvt_fingers(right)

In [11]:
# associate notes with synthesia metadata
lnotes = []
i = 0
for msg in mid.tracks[1]:
    if msg.type == 'note_on' and msg.dict()['velocity'] > 0:
        lnotes += [(msg,left[i])]
        i = i + 1
rnotes = []
i = 0
for msg in mid.tracks[0]:
    if msg.type == 'note_on' and msg.dict()['velocity'] > 0:
        rnotes += [(msg,right[i])]
        i = i + 1

In [12]:
# helper to get better information out of the dictionary 
def extract(v):
    arr = []
    for (msg,finger) in v:
        arr += [(msg.dict()['note'],msg.dict()['velocity'],finger)]
    return arr

In [13]:
lnotes_e,rnotes_e = extract(lnotes),extract(rnotes)

In [14]:
lnotes_e,rnotes_e

([(48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (43, 80, 5),
  (47, 80, 3),
  (50, 80, 1),
  (47, 80, 3),
  (50, 80, 1),
  (43, 80, 5),
  (47, 80, 3),
  (50, 80, 1),
  (47, 80, 3),
  (50, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (50, 80, 4),
  (52, 80, 3),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (50, 80, 4),
  (52, 80, 3),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (53, 80, 2),
  (57, 80, 1),
  (48, 80, 5),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80, 3),
  (55, 80, 1),
  (48, 80, 5),
  (52, 80, 3),
  (55, 80, 1),
  (52, 80,

In [15]:
# Attempt to group together accords


def group_accords(midi_msg_set):
    all_accords = []
    accord = []
    current = 0
    for msg, f in midi_msg_set:
        current += msg.time
        if msg.time > 0:
            t = 0
            entry = []
            for (note,finger,timing) in accord:
                entry += [(note,finger)]
                t = timing

            all_accords += [(entry,timing)]


            accord = [(msg.note,f,current)]
        else:
            accord += [(msg.note,f,current)]
    return all_accords

In [16]:
left_accords,right_accords = group_accords(lnotes),group_accords(rnotes)

In [17]:
left_accords,right_accords

([([(48, 5)], 0),
  ([(52, 3), (55, 1)], 25),
  ([(52, 3), (55, 1)], 278),
  ([(48, 5)], 531),
  ([(52, 3), (55, 1)], 556),
  ([(52, 3), (55, 1)], 809),
  ([(48, 5)], 1062),
  ([(52, 3), (55, 1)], 1087),
  ([(52, 3), (55, 1)], 1340),
  ([(48, 5)], 1593),
  ([(52, 3), (55, 1)], 1618),
  ([(52, 3), (55, 1)], 1871),
  ([(43, 5)], 2124),
  ([(47, 3), (50, 1)], 2149),
  ([(47, 3), (50, 1)], 2402),
  ([(43, 5)], 2655),
  ([(47, 3), (50, 1)], 2680),
  ([(47, 3), (50, 1)], 2933),
  ([(48, 5)], 3186),
  ([(52, 3), (55, 1)], 3211),
  ([(52, 3), (55, 1)], 3464),
  ([(48, 5)], 3717),
  ([(50, 4)], 3742),
  ([(52, 3)], 3767),
  ([(53, 2)], 3792),
  ([(57, 1)], 3817),
  ([(48, 5)], 3842),
  ([(53, 2)], 3867),
  ([(57, 1)], 3892),
  ([(48, 5)], 3917),
  ([(48, 5)], 3942),
  ([(52, 3), (55, 1)], 3967),
  ([(52, 3), (55, 1)], 4220),
  ([(48, 5)], 4473),
  ([(50, 4)], 4498),
  ([(52, 3)], 4523),
  ([(53, 2)], 4548),
  ([(57, 1)], 4573),
  ([(48, 5)], 4598),
  ([(53, 2)], 4623),
  ([(57, 1)], 4648),
  ([

In [18]:
preamble = [
    mido.Message('control_change',channel=0,control=121,value=0,time=0),
    mido.Message('program_change',channel=0,program=0,time=0),
    mido.Message('control_change',channel=0,control=7,value=100,time=0),
    mido.Message('control_change',channel=0,control=10,value=64,time=0),
    mido.Message('control_change',channel=0,control=91,value=0,time=0),
    mido.Message('control_change',channel=0,control=93,value=0,time=0)
]


In [19]:
import time
def play_single_accord(accord_data):
    data,_ = accord_data
    on_messages = []
    off_messages = []
    
    for (note,finger) in data:
        on_messages += [(mido.Message('note_on',note=note,velocity=80,channel=0,time=0),finger)]
        off_messages += [mido.Message('note_on',note=note,velocity=0, channel=0,time=1)]
    
    with mido.open_output(port) as output:
        for meta in preamble:
            output.send(meta)
        for on,finger in on_messages:
            print('Playing',on.note,'with finger',finger)
            output.send(on)
        time.sleep(1)
        for off in off_messages:
            output.send(off)
        
    

In [24]:
for i,accord in enumerate(left_accords):
    print(' --- Accord:',i)
    play_single_accord(accord)
    time.sleep(0.1)

 --- Accord: 0
Playing 48 with finger 5
 --- Accord: 1
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 2
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 3
Playing 48 with finger 5
 --- Accord: 4
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 5
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 6
Playing 48 with finger 5
 --- Accord: 7
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 8
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 9
Playing 48 with finger 5
 --- Accord: 10
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 11
Playing 52 with finger 3
Playing 55 with finger 1
 --- Accord: 12
Playing 43 with finger 5
 --- Accord: 13
Playing 47 with finger 3
Playing 50 with finger 1
 --- Accord: 14
Playing 47 with finger 3
Playing 50 with finger 1
 --- Accord: 15
Playing 43 with finger 5
 --- Accord: 16
Playing 47 with finger 3
Playing 50 with finger 1
 --- Accord: 17
Playing 47 with finger